## Ch3 MNIST_DNN

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

W0809 15:10:43.679089 139790327043904 deprecation.py:323] From <ipython-input-1-4001b7b9b605>:4: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0809 15:10:43.679624 139790327043904 deprecation.py:323] From /home/john/anaconda3/envs/tf_server/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0809 15:10:43.679942 139790327043904 deprecation.py:323] From /home/john/anaconda3/envs/tf_server/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated an

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [3]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_normal([784,256], stddev=0.01), name='W1')
    L1 = tf.nn.relu(tf.matmul(X,W1))
    
    tf.summary.histogram('weight_1', W1)
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_normal([256,256], stddev=0.01), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1,W2))
    
    tf.summary.histogram('weight_2', W2)
    
with tf.name_scope('layer3'):
    W3 = tf.Variable(tf.random_normal([256,10], stddev=0.01), name='W3')
    model = tf.matmul(L2,W3)
    
    tf.summary.histogram('weight_3', W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost, global_step=global_step)
    
    tf.summary.scalar('cost', cost)

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

W0809 15:10:44.193945 139790327043904 deprecation.py:323] From <ipython-input-4-14845d30eb9f>:7: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [5]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [6]:
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        summary = sess.run(merged, feed_dict={X: batch_xs, Y: batch_ys})
        writer.add_summary(summary, global_step=sess.run(global_step))
        
    print('Epoch: %04d' %(epoch+1),
          'Avg. cost: %.3f' % float(total_cost/total_batch))
    
saver.save(sess, './model/mnist_dnn.ckpt', global_step=global_step)
print("Done")

Epoch: 0001 Avg. cost: 0.397
Epoch: 0002 Avg. cost: 0.145
Epoch: 0003 Avg. cost: 0.095
Epoch: 0004 Avg. cost: 0.069
Epoch: 0005 Avg. cost: 0.052
Epoch: 0006 Avg. cost: 0.039
Epoch: 0007 Avg. cost: 0.032
Epoch: 0008 Avg. cost: 0.026
Epoch: 0009 Avg. cost: 0.022
Epoch: 0010 Avg. cost: 0.017
Done


In [7]:
mnist.test.images.shape

(10000, 784)

In [8]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

sess.close()

정확도:  0.9767


### 2. Dropout

In [9]:
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [10]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_normal([784,256], stddev=0.01), name='W1')
    L1 = tf.nn.relu(tf.matmul(X,W1))
    L1 = tf.nn.dropout(L1, keep_prob)
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_normal([256,256], stddev=0.01), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1,W2))
    L2 = tf.nn.dropout(L2, keep_prob)
    
with tf.name_scope('layer3'):
    W3 = tf.Variable(tf.random_normal([256,10], stddev=0.01), name='W3')
    model = tf.matmul(L2,W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost, global_step=global_step)

W0809 15:11:54.012493 139790327043904 deprecation.py:506] From <ipython-input-10-2cf25f26f7ce>:4: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model_dropout')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [12]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [13]:
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.8})
        total_cost += cost_val
        
    print('Epoch: %04d' %(epoch+1),
          'Avg. cost: %.3f' % float(total_cost/total_batch))
    
saver.save(sess, './model_dropout/mnist_dnn.ckpt', global_step=global_step)
print("Done")

Epoch: 0001 Avg. cost: 0.425
Epoch: 0002 Avg. cost: 0.165
Epoch: 0003 Avg. cost: 0.113
Epoch: 0004 Avg. cost: 0.087
Epoch: 0005 Avg. cost: 0.072
Epoch: 0006 Avg. cost: 0.059
Epoch: 0007 Avg. cost: 0.053
Epoch: 0008 Avg. cost: 0.046
Epoch: 0009 Avg. cost: 0.041
Epoch: 0010 Avg. cost: 0.037
Done


In [14]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.}))

sess.close()

정확도:  0.9795
